In [3]:
import os
%cd ..
os.getcwd()

/Users/macbook/Documents/Code/game


'/Users/macbook/Documents/Code/game'

In [4]:
import pygame
import importlib
import src.animation
importlib.reload(src.animation)
from src.animation import Animation
from src.utils import load_dict_from_folder

In [5]:
pygame.init()
WIDTH, HEIGHT = 1280, 720
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()
running = True
player_x, player_y = WIDTH // 2, HEIGHT // 2
player_size = 50 # 50 x 50
image_path = os.path.join("images", "backgrounds", "volcano.png")

background = pygame.image.load(image_path)
background = pygame.transform.scale(background, (WIDTH, HEIGHT))


In [130]:
player_dict = load_dict_from_folder("images/characters/mark")
player_dict

{'attack': <Surface(320x80x32 SW)>,
 'defend': <Surface(160x80x32 SW)>,
 'idle': <Surface(320x80x32 SW)>,
 'jump': <Surface(240x80x32 SW)>,
 'move': <Surface(320x80x32 SW)>,
 'run': <Surface(240x80x32 SW)>,
 'run_attack': <Surface(560x80x32 SW)>}

In [131]:
import pygame

class Player:
    def __init__(self, x, y, speed):
        self.x = x
        self.y = y
        self.base_speed = speed
        self.speed = speed  # Maintain current speed separately
        self.state = "idle"
        self.facing_right = True
        self.is_running = False
        self.last_press_d = 0
        self.last_press_a = 0

        # Jumping mechanics
        self.ground_y = 0
        self.is_jumping = False
        self.gravity = 1
        self.jump_velocity = -10  # Initial jump power

        # Attack & Defend mechanics
        self.is_attacking = False
        self.is_defending = False
        self.attack_timer = 0  # To reset state after attack
        self.defend_timer = 0

        self.animations = {
            "idle": Animation(player_dict["idle"], 300),
            "move": Animation(player_dict["move"], 150),
            "attack": Animation(player_dict["attack"], 200),
            "run": Animation(player_dict["run"], 150),
            "defend": Animation(player_dict["defend"], 300),
            "jump": Animation(player_dict["jump"], 200),
            "run_attack": Animation(player_dict["run_attack"], 150)
        }

    def update(self, keys, events):
        """Handles all input and updates player state"""
        current_time = pygame.time.get_ticks()

        # Handle key events (attack, defend, jump, run)
        for event in events:
            if event.type == pygame.KEYDOWN:
                keys = pygame.key.get_pressed()
                
                if event.key == pygame.K_d:
                    if current_time - self.last_press_d < 300:
                        self.start_running(direction="right")
                    self.last_press_d = current_time
                    
                elif event.key == pygame.K_a:
                    if current_time - self.last_press_a < 300:
                        self.start_running(direction="left")
                    self.last_press_a = current_time

                elif event.key == pygame.K_j:  # Attack
                    self.attack()

                elif event.key == pygame.K_k:  # Defend
                    self.defend()

                elif event.key == pygame.K_l:  # Jump
                    if not self.is_jumping:
                        self.jump()
                
           
        # Handle attack & defend cooldown

        
        if self.is_attacking and not self.is_running and current_time - self.attack_timer > 300:  # 300ms attack duration
            self.is_attacking = False
            self.state = "idle"

        elif self.is_attacking and self.is_running:
            if current_time - self.attack_timer > self.run_attack_duration:
                self.is_attacking = False
                self.is_running = False
                self.state = "idle"
                
        if self.is_defending and current_time - self.defend_timer > 1000:  # Defend lasts longer
            self.is_defending = False
            self.state = "idle"

        # Handle movement logic (if not attacking or defending)
        if not (self.is_attacking or self.is_defending or self.is_jumping):
            self.move(keys)

        # Handle jumping mechanics
        if self.is_jumping:
            self.state = "jump"
            self.y += self.jump_velocity
            self.jump_velocity += self.gravity
            if self.y >= self.ground_y:
                self.y = self.ground_y
                self.is_jumping = False
                self.state = "idle"

    def start_running(self, direction):
        self.is_running = True
        self.state = "run"
        self.speed = self.base_speed * 3
        self.facing_right = (direction == "right")
        
    def attack(self):
        """Initiates an attack"""
        if self.is_running:
            print("Press J, perform running attack")
            self.state = "run_attack"
            self.is_attacking = True
            self.attack_timer = pygame.time.get_ticks()
            self.run_attack_duration = self.animations["run_attack"].frame_num * self.animations["run_attack"].cooldown
            print(f"Running attack dur: {self.run_attack_duration}")
            
        elif not self.is_attacking:  # Prevent attack spam
            self.is_attacking = True
            self.state = "attack"
            self.attack_timer = pygame.time.get_ticks()

    def defend(self):
        """Enables defending state"""
        if not self.is_defending:  # Prevent spam
            self.is_defending = True
            self.state = "defend"
            self.defend_timer = pygame.time.get_ticks()

    def jump(self):
        """Initiates jump"""
        self.ground_y = self.y
        self.is_jumping = True
        self.state = "jump"
        self.jump_velocity = -10  # Jump impulse

    def move(self, keys):
        """Handles movement if not attacking or defending"""
        moving = False

        if self.is_running:
            if (self.facing_right and keys[pygame.K_a]) or (not self.facing_right and keys[pygame.K_d]):
                moving = False
                self.is_running = False
                self.state = "idle"
            
            elif self.facing_right:    
                if self.x + 80 + self.speed < WIDTH:
                    self.x += self.speed
                    moving = True
            
            elif not self.facing_right:
                if self.x - self.speed > 0:
                    self.x -= self.speed
                    moving = True
            
            if keys[pygame.K_w]:
                if self.y - self.base_speed > 0:
                    self.y -= self.base_speed
                moving = True

            if keys[pygame.K_s]:
                if self.y + 80 + self.base_speed < HEIGHT:
                    self.y += self.base_speed
                moving = True

        if not self.is_running:
            if keys[pygame.K_d]:
                self.facing_right = True
                if self.x + 80 + self.base_speed < WIDTH:
                    self.x += self.base_speed
                moving = True

            if keys[pygame.K_a]:
                self.facing_right = False
                if self.x - self.base_speed > 0:
                    self.x -= self.base_speed
                moving = True
                
            if keys[pygame.K_w]:
                if self.y - self.base_speed > 0:
                    self.y -= self.base_speed
                moving = True

            if keys[pygame.K_s]:
                if self.y + 80 + self.base_speed < HEIGHT:
                    self.y += self.base_speed
                moving = True

        if moving:
            self.state = "run" if self.is_running else "move"
        else:
            self.state = "idle"
            self.is_running = False

    def draw(self, screen):
        """Draw the player on screen"""
        frame = self.animations[self.state].get_current_frame()
        if not self.facing_right:
            frame = pygame.transform.flip(frame, True, False)
        screen.blit(frame, (self.x, self.y))


In [136]:
player = Player(100, 100, speed=2)

background = pygame.image.load("images/backgrounds/good_back.jpg")
background = pygame.transform.scale(background, screen.get_size())

running = True
while running:
    screen.blit(background, (0, 0))  
    keys = pygame.key.get_pressed()
    events = pygame.event.get()
    
    for event in events:
        if event.type == pygame.QUIT:
            running = False
    
    player.update(keys, events)
    print(f"Position: {player.x, player.y}")
    player.draw(screen)
    pygame.display.flip()
    clock.tick(60) 


Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (100, 100)
Position: (10

KeyboardInterrupt: 

In [120]:
from src.utils import cutting_sprite
cutting_sprite("switch", "run_attack", pos=[[1, 5], [1, 6], [1, 7]])

Saved!
